# Calculate [user x item] array B for WMF

In [1]:
import os
import collections
from scipy.sparse import csr_matrix
from sys import getsizeof
import numpy as np
import pickle

In [5]:
def create_user_item_matrix(filereader, max_lines=1000):
    user2i = collections.defaultdict(lambda: len(user2i)) # user to index dict
    item2i = collections.defaultdict(lambda: len(item2i)) # item to index dict

    rows = [] # row indices (users) for sparse matrix
    cols = [] # col indices (items) for sparse matrix
    counts = [] # values present at indices given by rows, cols

    for i, line in enumerate(f):
        user, item, count = line.split()
        user_index = user2i[user]
        item_index = item2i[item]
        rows.append(int(user_index))
        cols.append(int(item_index))
        counts.append(int(count))
        if i == max_lines: break
    
    user_item_matrix = csr_matrix((np.array(counts), (np.array(rows), np.array(cols))))
    return user_item_matrix

with open('data/taste_profile_subset/train_triplets.txt', encoding='utf8') as f:
    user_item_matrix = create_user_item_matrix(f, max_lines=None)

In [6]:
print(user_item_matrix.shape)
# pickle.dump(user_item_matrix, open('../user_item_matrix.pkl', 'wb'))

(1019318, 384546)


In [7]:
from wmf import wmf
pkl_file = open('../user_item_matrix.pkl', 'rb')
B = pickle.load(pkl_file)
S = wmf.log_surplus_confidence_matrix(B, alpha=2.0, epsilon=1e-6)
U, V = wmf.factorize(S, num_factors=41, lambda_reg=1e-5, num_iterations=2, init_std=0.01, verbose=True, dtype='float32', recompute_factors=wmf.recompute_factors_bias)

ModuleNotFoundError: No module named 'wmf'

In [6]:
pickle.dump(U, open('../user_wmf.pkl', 'wb'))
pickle.dump(V, open('../item_wmf.pkl', 'wb'))